# __Search Text by Text__

- Tutorial Difficulty : ★★☆☆☆
- 7 min read 
- Languages : [SQL](https://en.wikipedia.org/wiki/SQL) (100%)
- File location : tutorial_en/thanosql_search/search_text_by_text.ipynb   
- References : [(Kaggle) IMDB Movie Reviews](https://www.kaggle.com/code/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews/data), [Worrd Embeddings: LEXICAL SEMANTICS Encoding](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

## Tutorial Introduction

<div class="admonition note">
    <h4 class="admonition-title">Understanding Text Vectorization</h4>
    <p>Computers cannot directly interpret human language (natural language). Therefore, a process for converting natural language into numerical data that can be recognized by computers is required. In the field of natural language processing, embedding refers to the result of converting human natural language into a vectorized format, which is a form that can be understood by machines.</p>
</div>

Techniques for converting natural language into embeddings are largely divided into statistical techniques and artificial neural network-based techniques. ThanoSQL provides a method to train a text vectorization model using self-supervised learning.

<div class="admonition note">
    <h4 class="admonition-title">In this tutorial</h4>
    <p>👉 Uses <mark style="background-color:#FFD79C">movie review data</mark>. The data consists of movie review texts and label values. However, because we are demonstrating self-supervised learning, this tutorial does not use label values. By training a model with 4,000 movie reviews, we will be able to search text by text and extract the importance of each word from a given movie review. </p>
</div>

## __0. Prepare Dataset__

As mentioned in the [ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/how_to_use_ThanoSQL/#5-thanosql-workspace), you must create an API token and run the query below to execute the query of ThanoSQL. 

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<Issued_API_TOKEN>

### __Prepare Dataset__

In [2]:
%%thanosql
GET THANOSQL DATASET movie_review_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" downloads the specified dataset to the workspace. </li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>GET THANOSQL DATASET</strong> clause.
        <ul>
            <li>"overwrite" : Determines whether to overwrite a dataset if it already exists. If set as True, the old dataset is replaced with the new dataset (True|False, DEFAULT : False) </li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY movie_review_train
OPTIONS (overwrite=True) 
FROM "thanosql-dataset/movie_review_data/movie_review_train.csv"

Success


In [4]:
%%thanosql
COPY movie_review_test 
OPTIONS (overwrite=True) 
FROM "thanosql-dataset/movie_review_data/movie_review_test.csv"

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>COPY</strong>" specifies the name of the dataset to be saved as a database table. </li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>COPY</strong> clause.
        <ul>
           <li>"overwrite" : Determines whether to overwrite a table if it already exists. If set as True, the old table is replaced with the new table (True|False, DEFAULT : False) </li>
        </ul>
        </li>
    </ul>
</div>

## __1. Check Dataset__

To create a movie review text search model, we use the <mark style="background-color:#FFEC92 ">movie_review_train</mark> table from the ThanoSQL workspace database. To check the table's contents, run the following query.

In [5]:
%%thanosql
SELECT *
FROM movie_review_train
LIMIT 5 

,review,sentiment
0,This is the kind of movie that BEGS to be show...,negative
1,Bulletproof is quite clearly a disposable film...,negative
2,A beautiful shopgirl in London is swept off he...,positive
3,"VERY dull, obvious, tedious Exorcist rip-off f...",negative
4,Do we really need any more narcissistic garbag...,negative


<div class="admonition note">
   <h4 class="admonition-title">Understanding the Data</h4>
   <ul>
      <li><mark style="background-color:#D7D0FF ">review</mark>: movie review in text format</li>
      <li><mark style="background-color:#D7D0FF ">sentiment</mark> : target value indicating whether the review has a positive or negative sentiment</li>
   </ul>
</div>

## __2. Create a Text Vectorization Model__

To create a text search model with the name <mark style="background-color:#E9D7FD ">movie_text_search_model</mark> using the <mark style="background-color:#FFEC92 ">movie_review_train</mark> dataset, run the following query.  
(Estimated time required for query execution: 2 min)

In [6]:
%%thanosql
BUILD MODEL movie_text_search_model
USING SBERTEn
OPTIONS (
    text_col="review",
    overwrite=True
)
AS
SELECT *
FROM movie_review_train

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" creates and trains a model named <mark style="background-color:#E9D7FD">movie_text_search_model</mark>.</li>
        <li>"<strong>USING</strong>" specifies <mark style="background-color:#E9D7FD">SBERTEn</mark> as the base model.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values used to create a model.
        <ul>
            <li>"text_col" : a column containing movie review data in the data table. (Default : "text")</li>
            <li>"epochs" : number of times to train with the training dataset (int, DEFAULT : 1)</li>
            <li>"batch_size" : the size of dataset bundle utilized in a single cycle of training. (int, DEFAULT : 16)</li> 
            <li>"learning_rate" : the learning rate of the model (float, DEFAULT : 3e-5)</li> 
            <li>"train" : determines whether to use a pretrained model. If specified as False, the pretrained model is used. (True|False, DEFAULT : True)</li> 
            <li>"overwrite": determines whether to overwrite a model if it already exists. If set as True, the old model is replaced with the new model (True|False, DEFAULT : False)</li>
        </ul>
        </li>
    </ul>
</div>

To vectorize the `movie_review_test` texts run the following "__CONVERT USING__" query. The vectorized results are stored in a user-defined column (default: <mark style="background-color:#D7D0FF">convert_result</mark>) in the `movie_review_test` table.

In [7]:
%%thanosql
CONVERT USING movie_text_search_model
OPTIONS (
    text_col="review",
    table_name="movie_review_test",
    batch_size=32,
    column_name="convert_result"
    )
AS 
SELECT *
FROM movie_review_test

,review,sentiment,convert_result
0,"I read the book before seeing the movie, and t...",positive,"[0.014037576, 0.027578924, 0.02471304, -0.0176..."
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,"[0.027382538, -0.021728337, 0.03685805, -0.044..."
2,"Yesterday I attended the world premiere of ""De...",positive,"[0.014439536, -0.02376758, -0.0065942346, 0.03..."
3,Moonwalker is a Fantasy Music film staring Mic...,positive,"[0.012731184, 0.020806227, 0.0311054, -0.00222..."
4,"Welcome to Oakland, where the dead come out to...",positive,"[-0.0044349744, -0.06204289, 0.017439988, -0.0..."
...,...,...,...
995,Ocean's 12 starts off on annoying and gets wor...,negative,"[0.02004037, 0.015577145, 0.0042112377, -0.051..."
996,I remember catching this movie on one of the S...,negative,"[0.016599433, 0.008860707, 0.007274589, 0.0302..."
997,CyberTracker is set in Los Angeles sometime in...,negative,"[-0.0038948876, 0.033606753, 0.02673663, 0.000..."
998,"There is so much that is wrong with this film,...",negative,"[0.00040617603, 0.03672079, -0.010704152, -0.0..."


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" uses <code>movie_text_search_model</code> as an algorithm for text vectorizaion.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for text vectorizaion.
        <ul>
            <li>"text_col" : a column containing movie review data in the data table. (Default : "text")</li>
            <li>"table_name" : the table name to be stored in the ThanoSQL workspace database.</li>
            <li>"batch_size" : the size of dataset bundle utilized in a single cycle of training. (int, DEFAULT : 16)</li>
            <li>"column_name" : defines the column name that contains the vectorized results. (default: "convert_result")</li>
        </ul>
        </li>
    </ul>
</div>

## __3. Search for Similar Texts__

This step uses the <mark style="background-color:#E9D7FD">movie_text_search_model</mark> text vectorization model and test table to search for similar texts.

In [8]:
%%thanosql
SELECT review, sentiment, score
FROM (
    SEARCH TEXT text="This movie was my favorite movie of all time"
    USING movie_text_search_model
    OPTIONS (
        emb_col="convert_result",
        column_name="score"
        )
    AS 
    SELECT * 
    FROM movie_review_test
    )
ORDER BY score DESC 
LIMIT 10

,review,sentiment,score
0,I have loved this movie since I saw it in the ...,positive,0.741026
1,"I always loved this film. The music,story and ...",positive,0.735022
2,I saw this movie for the first time in 1988 wh...,positive,0.734215
3,This movie is Jackie's best. I still cant get ...,positive,0.731211
4,This is a delightful film. Elizabeth Taylor do...,positive,0.728850
5,This movie was the second movie I saw on the c...,positive,0.716220
6,Very heart warming and uplifting movie. Outsta...,positive,0.706558
7,My husband and I both loved this film. At firs...,positive,0.702175
8,This has to be one of the best movies to come ...,positive,0.700377
9,What can I say?? This movie has it all...Roman...,positive,0.697864


In [9]:
%%thanosql
SELECT review, sentiment, score
FROM (
    SEARCH TEXT text="The movie was unsatisfactory"
    USING movie_text_search_model
    OPTIONS (
        emb_col="convert_result",
        column_name="score"
        )
    AS 
    SELECT * 
    FROM movie_review_test
    )
ORDER BY score DESC 
LIMIT 10

,review,sentiment,score
0,Just plain terrible. Nick and Michael are WAY ...,negative,0.745355
1,There was absolutely nothing in this film that...,negative,0.745163
2,"A gave it a ""2"" instead of a ""1"" (awful) becau...",negative,0.744775
3,"We brought this film as a joke for a friend, a...",negative,0.742644
4,"I knew this movie wasn't going to be amazing, ...",negative,0.733149
5,This movie looked fun on the cover and I hones...,negative,0.730956
6,Badly made. Dreadful acting and an ending that...,negative,0.728068
7,"This movie was so unrelentingly bad, I could h...",negative,0.724352
8,"The premise of this movie was decent enough, b...",negative,0.722977
9,"While the dog was cute, the film was not. It w...",negative,0.722429


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|audio|video|text|keyword]</strong>" defines the image|audio|video|text|keyword data type to search for.</li>
        <li>"<strong>USING</strong>" defines the model used for the text vectorization.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for text searching.
        <ul>
                <li>"emb_col" : the column that contains the vectorized results.</li>
                <li>"column_name" : defines the name of the column that contains the search results. (default: "search_result")</li>
        </ul>
        </li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for the searches. In this example, <code>movie_review_test</code> table is used.</li>
    </ul>
</div>

## __4. Extract Keywords from Texts__

This step uses the <mark style="background-color:#E9D7FD">movie_text_search_model</mark> text vectorization model and test table to extract keywords from the texts.

In [10]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col="review",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS 
SELECT * 
FROM movie_review_test
LIMIT 10 OFFSET 40

,review,sentiment,convert_result,keyword
0,"Like an earlier commentor, I saw it in 1980 an...",positive,"[0.019816624, -0.033710007, -0.01097258, -0.03...","{'keyword': ['story splendid acting', 'writers..."
1,"I'm a fan of Matthew Modine, but this film--wh...",negative,"[0.004325681, 0.015616861, -0.0028261256, -0.0...","{'keyword': ['film', 'i watched it', 'all this..."
2,Paul Lukas played a Russian intellectual makin...,positive,"[0.034945264, -0.03700541, 0.004724759, -0.047...","{'keyword': ['comedies of preston', 'celebrity..."
3,"to be honest, i didn't watch all of the origin...",negative,"[0.03015102, 0.013378834, 0.002104831, 0.00672...","{'keyword': ['other vampire movie', 'horror ha..."
4,Police Squad! (1982) was a funny show that end...,positive,"[0.00814611, -0.014099172, 0.018553793, -0.013...","{'keyword': ['a funny show', 'comedian in this..."
5,I am still shuddering at the thought of EVER s...,negative,"[0.0071278927, 0.030981911, 0.006835275, -0.02...","{'keyword': ['seeing this movie', 'but the plo..."
6,Gregory Peck gives a brilliant performance in ...,positive,"[0.034372132, 0.0354431, -0.02025241, -0.06389...","{'keyword': ['a brilliant performance', 'the f..."
7,I first flicked onto the LoG accidentally one ...,positive,"[-5.251513e-05, 0.03718008, 0.008339626, -0.00...","{'keyword': ['t watch it', 'an amazing plot', ..."
8,I didn't know what to make of this film. I gue...,positive,"[-0.004017977, 0.018181568, 0.0027857828, -0.0...","{'keyword': ['view the film', 'think i will', ..."
9,A family looking for some old roadside attract...,negative,"[0.0325733, -0.010194275, -0.012354068, -0.038...","{'keyword': ['poorly made horror', 'beloved fr..."


In [11]:
%%thanosql
SELECT review, sentiment, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD 
    USING movie_text_search_model
    OPTIONS (
        text_col="review",
        use_stopwords=True
        )
    AS 
    SELECT * 
    FROM movie_review_test
    LIMIT 10
)

,review,sentiment,keywords,score
0,"I read the book before seeing the movie, and t...",positive,"[best adaptations, the movie, talented filmake...","[0.5384, 0.5241, 0.4566, 0.4392, 0.3636]"
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,"[9/11 hosted, television show, 2001, robert de...","[0.6425, 0.4782, 0.4376, 0.3801, 0.306]"
2,"Yesterday I attended the world premiere of ""De...",positive,"[rape scene, she is, little hesitant, become s...","[0.4252, 0.266, 0.2627, 0.2198, 0.1661]"
3,Moonwalker is a Fantasy Music film staring Mic...,positive,"[music film, song bad, i liked, 6/10 segment, ...","[0.5701, 0.5011, 0.4273, 0.362, 0.3005]"
4,"Welcome to Oakland, where the dead come out to...",positive,"[supernatural clich, brother jermaine, the ghe...","[0.4682, 0.4022, 0.3658, 0.3223, 0.2118]"
5,Tipping the Velvet (2002) (TV) was directed by...,positive,"[protagonist nan, excellent actor, rachael sti...","[0.4539, 0.3763, 0.3458, 0.3283, 0.2516]"
6,The Stock Market Crash of 1929 and the Depress...,positive,"[james cagney, between musicals, screened his,...","[0.504, 0.4535, 0.3151, 0.3124, 0.2208]"
7,I want to clarify a few things. I am not famil...,negative,"[art movies, undeserved commercial, is boring,...","[0.5926, 0.3547, 0.3486, 0.3375, 0.2642]"
8,This is a nice movie with good performances by...,positive,"[better movie, spanish cinema, is very, leonar...","[0.5518, 0.5473, 0.3112, 0.216, 0.0852]"
9,"Once a month, I invite a few friends over for ...",negative,"[retarded movie, standard zombie, friends howe...","[0.5442, 0.2274, 0.2009, 0.1933, 0.1527]"


In [12]:
%%thanosql 
SELECT * FROM (
    SELECT review, sentiment, json_array_elements(keyword -> 'keyword') AS keywords, (json_array_elements(keyword -> 'score'))::text::float AS score
        FROM (
            SEARCH KEYWORD 
            USING movie_text_search_model
            OPTIONS (
                text_col="review",
                use_stopwords=True
                )
            AS 
            SELECT * 
            FROM movie_review_test
            LIMIT 10
        )
    ) 
WHERE score > 0.5

,review,sentiment,keywords,score
0,"I read the book before seeing the movie, and t...",positive,best adaptations,0.5384
1,"I read the book before seeing the movie, and t...",positive,the movie,0.5241
2,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,9/11 hosted,0.6425
3,Moonwalker is a Fantasy Music film staring Mic...,positive,music film,0.5701
4,Moonwalker is a Fantasy Music film staring Mic...,positive,song bad,0.5011
5,This is a nice movie with good performances by...,positive,better movie,0.5518
6,The Stock Market Crash of 1929 and the Depress...,positive,james cagney,0.5040
7,I want to clarify a few things. I am not famil...,negative,art movies,0.5926
8,This is a nice movie with good performances by...,positive,spanish cinema,0.5473
9,"Once a month, I invite a few friends over for ...",negative,retarded movie,0.5442


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH KEYWORD</strong>" uses the movie_text_search_model as an algorithm to search keywords.</li>
        <li>"<strong>USING</strong>" defines the model used for the text vectorization.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for the text vectorizaion.
            <ul>
                <li>"lang" : en, ko</li>
                <li>"text_col" :  a column containing movie review data in the data table</li>
                <li>"ngram_range" : minimum and maximum number of words for each keyword. ex) [1, 3]. In most situations, keywords are extracted according to the maximum number of words. (list[int, int], DEFAULT : [1, 2])</li>
                <li>"top_n" : number of keywords to be extracted, in order of highest similarity (int, DEFAULT : 5)</li>
                <li>"diversity" : variety of keywords to be extracted. The higher the value, the more diverse the keywords will be. 0 <= diversity <= 1 (float, DEFAULT : 0.5)</li>
                <li>"use_stopwords" : whether to exclude words that do not have a significant meaning (True|False, DEFAULT : True)</li>
                <li>"threshold" : minimum value of similarity value of keywords to be extracted. (float, DEFAULT : 0.0)</li>
            </ul>
            </li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, <code>movie_review_test</code> table is used.</li>
    </ul>
</div>

## __5. Combine the Two Methods__

In [13]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col="review",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT review, sentiment, score
    FROM (
        SEARCH TEXT text="The greatest movie of all time"
        USING movie_text_search_model
        OPTIONS (
            emb_col="convert_result",
            column_name="score"
            )
        AS 
        SELECT * 
        FROM movie_review_test
        )
    ORDER BY score DESC 
    LIMIT 10
)

,review,sentiment,score,keyword
0,This is without a doubt the most poorly though...,negative,0.656462,"{'keyword': ['movie in history', 'bad as this'..."
1,This is a delightful film. Elizabeth Taylor do...,positive,0.655172,"{'keyword': ['recommend this film', 'delightfu..."
2,The beginning of this film is a little clunky ...,positive,0.654622,"{'keyword': ['thriller', 'film is', 'fine perf..."
3,"First ever viewing: July 21, 2008 Very impress...",positive,0.653558,"{'keyword': ['best adapted screenplay', 'rarel..."
4,This is without a doubt the greatest film ever...,positive,0.647122,"{'keyword': ['greatest film ever', 'improvised..."
5,This is the best Chinese movie I have ever see...,positive,0.645870,"{'keyword': ['best chinese movie', 'movie defi..."
6,"This film may seem dated today, but remember t...",positive,0.643857,"{'keyword': ['made in 1974', 'heavy satiric hu..."
7,This movie is Jackie's best. I still cant get ...,positive,0.640296,"{'keyword': ['movie is jackie', 'great story',..."
8,Devil Dog sets your heart racing. It's brillia...,negative,0.634332,"{'keyword': ['brilliantly paced', 'the ending'..."
9,This movie was the second movie I saw on the c...,positive,0.631678,"{'keyword': ['movie great monsters', 'truly ro..."


In [14]:
%%thanosql
SELECT review, sentiment, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM ( 
    SEARCH KEYWORD
    USING movie_text_search_model
    OPTIONS (
        text_col="review",
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT review, sentiment, score
        FROM (
            SEARCH TEXT text="Such a romatic movie"
            USING movie_text_search_model
            OPTIONS (
                emb_col="convert_result",
                column_name="score"
                )
            AS 
            SELECT * 
            FROM movie_review_test
            )
        ORDER BY score DESC 
        LIMIT 10
    )
)

,review,sentiment,keywords,score
0,i would have to say that this is the first qua...,positive,"[quality romantic comedy, movie was well, stil...","[0.6767, 0.6377, 0.2868, 0.2609, 0.221]"
1,Burlinson and Thornton give an outstanding per...,positive,"[the horse scenes, love this movie, are absolu...","[0.6556, 0.6132, 0.4957, 0.41, 0.1869]"
2,Very heart warming and uplifting movie. Outsta...,positive,"[uplifting movie outstanding, saw this movie, ...","[0.6771, 0.5455, 0.4628, 0.2198, 0.1636]"
3,"Modern, original, romantic story. Very good ac...",positive,"[very good acting, original romantic story, go...","[0.5996, 0.5457, 0.4167, 0.2933, 0.2207]"
4,I came across this movie back in the mid eight...,positive,"[great christmas romance, film he, stanwyck is...","[0.6426, 0.4323, 0.4279, 0.3628, 0.3341]"
5,Forget the recent dire American remake which s...,positive,"[the intricate screenplay, deftly drawn romant...","[0.6248, 0.5623, 0.4376, 0.3847, 0.3049]"
6,My husband and I both loved this film. At firs...,positive,"[loved this film, wonderfully loveable and, hu...","[0.6392, 0.3914, 0.242, 0.1692, 0.1441]"
7,This is a delightful film. Elizabeth Taylor do...,positive,"[recommend this film, delightful film elizabet...","[0.7081, 0.6414, 0.4529, 0.3554, 0.2257]"
8,I have loved this movie since I saw it in the ...,positive,"[loved this movie, movie it is, dating loves i...","[0.5947, 0.498, 0.2693, 0.2514, 0.1677]"
9,Dani(Reese Witherspoon) has always been very c...,positive,"[her first movie, heartbreaking story about, a...","[0.5191, 0.4982, 0.4643, 0.3913, 0.223]"


In [15]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col="review",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT review, sentiment, score
    FROM (
        SEARCH TEXT text="The best action movie"
        USING movie_text_search_model
        OPTIONS (
            emb_col="convert_result",
            column_name="score"
            )
        AS 
        SELECT * 
        FROM movie_review_test
        WHERE review LIKE '%%gun%%'
        )
    ORDER BY score DESC 
    LIMIT 10
)

,review,sentiment,score,keyword
0,What was there about 1939 that helped produce ...,positive,0.581589,"{'keyword': ['excellent hollywood films', 'lon..."
1,This is quite an unusual and unique little wes...,positive,0.571282,"{'keyword': ['very entertaining movie', 'no cr..."
2,As a veteran screen writing instructor at Rich...,positive,0.541819,"{'keyword': ['truly good movies', 'before holl..."
3,Countless Historical & cultural mistakes 0/10 ...,negative,0.539601,"{'keyword': ['top 250 movies', 'movie 3 jewish..."
4,"Oh, dear! This has to be one of the worst film...",negative,0.516846,"{'keyword': ['the worst films', 'unbelievably ..."
5,This is a very memorable spaghetti western. It...,positive,0.503518,"{'keyword': ['poorly choreographed and', 'movi..."
6,"""In the world of old-school kung fu movies, wh...",positive,0.486575,"{'keyword': ['extraordinary kung fu', 'his lat..."
7,"This was probably the worst movie ever, seriou...",negative,0.484984,"{'keyword': ['worst movie ever', 'cheesy porn ..."
8,I first saw this as a child living in East Lon...,positive,0.480955,"{'keyword': ['history in cinema', 'space trave..."
9,I was very surprised how bad this movie was. N...,negative,0.456414,"{'keyword': ['kung fu movies', 'movie is worse..."


In [16]:
%%thanosql
SELECT * 
FROM (
    SELECT review, sentiment, json_array_elements(keyword -> 'keyword') AS keywords, (json_array_elements(keyword -> 'score'))::text::float AS score
    FROM (
        SEARCH KEYWORD
        USING movie_text_search_model
        OPTIONS (
            text_col="review",
            ngram_range=[1, 3],
            use_stopwords=True
            )
        AS (
            SELECT review, sentiment, score
            FROM (
                SEARCH TEXT text="The best action movie"
                USING movie_text_search_model
                OPTIONS (
                    emb_col="convert_result",
                    column_name="score"
                    )
                AS 
                SELECT * 
                FROM movie_review_test
                WHERE review LIKE '%%gun%%'
                )
            ORDER BY score DESC 
            LIMIT 10
        )
    )
)
WHERE score > 0.3

,review,sentiment,keywords,score
0,What was there about 1939 that helped produce ...,positive,excellent hollywood films,0.6745
1,What was there about 1939 that helped produce ...,positive,long forgotten screwball,0.4139
2,What was there about 1939 that helped produce ...,positive,the wedding,0.3555
3,This is quite an unusual and unique little wes...,positive,very entertaining movie,0.6288
4,This is quite an unusual and unique little wes...,positive,no criticism really,0.4036
5,This is quite an unusual and unique little wes...,positive,actors rock hudson,0.3810
6,This is quite an unusual and unique little wes...,positive,tough gunslinger seems,0.3562
7,As a veteran screen writing instructor at Rich...,positive,truly good movies,0.5596
8,As a veteran screen writing instructor at Rich...,positive,before hollywood started,0.4226
9,Countless Historical & cultural mistakes 0/10 ...,negative,top 250 movies,0.5378


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|audio|video|text|keyword]</strong>" defines the image|audio|video|text|keyword data type to search for.</li>
        <li>"<strong>SEARCH KEYWORD</strong>" uses the movie_text_search_model as an algorithm to search keywords.</li>
        <li>"<strong>USING</strong>" defines the model used for text vectorization.</li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, <code>movie_review_test</code> table is used.</li>
    </ul>
</div>

## __6. In Conclusion__

In this tutorial, we performed text vectorization using `movie review data`, and similar text search and keyword extraction. As this is a beginner-level tutorial, we focused on the process rather than accuracy. The model's accuracy can be improved by adjusting various options, such as increasing the epoch or dataset size. Create your own model and provide competitive services by combining various unstructured data (image, audio, video, etc.) and structured data with ThanoSQL.
<br>
For the next step, explore the various "OPTIONS" and training methods of text vectorization models. If you want to learn more about building your own text model, proceed with the following tutorials.

* [How to Upload to ThanoSQL DB](https://docs.thanosql.ai/en/getting_started/data_upload/)
* [Creating an Intermediate Similar Text Search Model]

<div class="admonition tip">
    <h4 class="admonition-title">Inquiries about deploying a model for your own service</h4>
    <p>If you have any difficulties creating your own model using ThanoSQL or applying it to your services, please feel free to contact us below😊</p>
    <p>For inquiries regarding building an text similarity search model: contact@smartmind.team</p>
</div>